<a href="https://colab.research.google.com/github/lamoon/DHT-sensor-library/blob/master/01_prep_rainfall_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import datetime
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from ftplib import FTP

In [ ]:

def ftp_connection(ftp_address, user, password):
    '''
    Create a FTP connection
    '''
    
    ftp_conn = FTP(FTP_ADDRESS)  # connect to host, default port
    #ftp.connect(port=21)       
    ftp_conn.login(user=USER, passwd=PASSWORD)# user anonymous, passwd anonymous@


    return ftp_conn


def ftp_walk(ftp_conn, dir, df_ftp_path):
    '''
    List all files (recursively) and save into df dataframe. df is required to define as global object.
    '''
    dirs = []
    nondirs = []
    for item in ftp_conn.mlsd(dir):
        if item[1]['type'] == 'dir':
            dirs.append(item[0])            
        else:            
            if item[0] not in ('.', '..'):
                nondirs.append(item[0])
                id = len(df_ftp_path)
                df_ftp_path.at[id, 'dir'] = dir
                df_ftp_path.at[id, 'filename'] = item[0]
                str_mod_date = str(item[1]['modify'])[:14]          
                #df.at[id, 'mod_date'] = datetime.strptime(str_mod_date, '%Y%m%d')
                df_ftp_path.at[id, 'mod_date'] = datetime.strptime(str_mod_date, '%Y%m%d%H%M%S') + timedelta(hours=7)
    if nondirs:        
        print('{} : {} files.'.format(dir,len(nondirs)))
        #print('\n'.join(sorted(nondirs)))
        
    else:
        # print(dir, 'is empty')
        pass
    for subdir in sorted(dirs):
        ftp_walk(ftp_conn, '{}/{}'.format(dir, subdir), df_ftp_path)

        

def ftp_download(ftp_conn, ftp_path, out_path, skip_exist=True):
    '''
    Download file from specified ftp path.
	'''
    if (os.path.exists(out_path)==True) & (skip_exist==True):
        print(f'{out_path} already exists.')
        return
    os.makedirs(os.path.dirname(out_path), exist_ok=True)      
    try:
        with open(out_path, 'wb') as fp:        
            ftp_conn.retrbinary(f'RETR {ftp_path}', fp.write)
            print(f'{ftp_path} has been downloaded.')
    except Exception as err:
        print(err)
        return        

    


In [ ]:
import re
def list_files_re(rootpath, filename_re=None, folder_re=None ):
    '''
    rootpath : root path to lookup files
    filename_re : regular expression to search for filename
    folder_re : regular expression to search for folder

    return : a list of filepaths
    '''
    list_files = []
    for folder, _, files in os.walk(rootpath):
        for file in files:     
            if filename_re == None:
                filename_re = '.*'
            if folder_re == None:
                folder_re = '.*'
                
            if ((re.search(filename_re, file) != None) & (re.search(folder_re, folder) != None)):
                list_files.append(os.path.join(folder, file))
        
    return list_files    

## download


In [ ]:
folder =  'thaiwa/rainfall'

tmp_folder = os.path.join(folder, 'tmp')
download_folder = os.path.join(folder, 'downloaded')
raw_folder = os.path.join(folder, 'raw')

os.makedirs(tmp_folder, exist_ok=True)   
os.makedirs(download_folder, exist_ok=True)   
os.makedirs(raw_folder, exist_ok=True)   

In [ ]:
MAIN_FOLDER = download_folder
TMP_FOLDER = tmp_folder
FTP_ADDRESS = 'hokusai.eorc.jaxa.jp'
USER = 'rainmap'
PASSWORD = 'Niskur+1404'

ftp_conn = FTP(FTP_ADDRESS, USER, PASSWORD)

START_DATE = f'20210601'
END_DATE = f'20210626'

st_date = datetime.datetime.strptime(START_DATE, '%Y%m%d')
end_date = datetime.datetime.strptime(END_DATE, '%Y%m%d')
no_of_days_between = (end_date - st_date).days
copy_frequency = 2

list_dates = [st_date + datetime.timedelta(days=i) for i in range(0, no_of_days_between)]
list_hours = [f'{hour:04d}' for hour in np.arange(0, 2400, 100)]

ftp_conn = ftp_connection(FTP_ADDRESS, USER, PASSWORD)
ftp_root_path = '/realtime_ver/v6/txt/02_AsiaSE'


In [ ]:
list_files = []
count_days = 0
for str_date in tqdm(list_dates):
    yyyymmdd = datetime.datetime.strftime(str_date, '%Y%m%d')
    processed_path = os.path.join(raw_folder, yyyymmdd[:4], f'gsmap_rt_v6_thailand_{yyyymmdd}.parquet')
    if os.path.exists(processed_path):
        print(processed_path, 'continue.')
        continue
        
    for str_hour in list_hours:
        filename = f'gsmap_nrt.{yyyymmdd}.{str_hour}.02_AsiaSE.csv.zip'
        ftp_path = os.path.join(ftp_root_path, yyyymmdd[:4], yyyymmdd[4:6], yyyymmdd[6:8], filename)    
        tmp_out_path = os.path.join(TMP_FOLDER, ftp_path[1:])  
        dst_path = tmp_out_path.replace(TMP_FOLDER, MAIN_FOLDER) 

        #If no file in temp & drive
        if (os.path.exists(dst_path) == False) & (os.path.exists(tmp_out_path) == False):
            try:     
                for attempt in range(0, 10):
                    ftp_download(ftp_conn, ftp_path, tmp_out_path, skip_exist=True)
                    if os.path.getsize(tmp_out_path) > 0:
                        list_files.append(tmp_out_path)
                        break
                    else:
                        pass
#                         print(f'Attemp {attempt+1} : Failed to download {ftp_path}.')                       
            except Exception as err :
                print(err)

        #if the file is existing in temp
        elif os.path.exists(tmp_out_path)==True:
            if os.path.getsize(tmp_out_path) > 0:
                list_files.append(tmp_out_path)
            else:
                os.remove(tmp_out_path)
                for attempt in range(0, 10):
                    ftp_download(ftp_conn, ftp_path, tmp_out_path, skip_exist=True)
                    if os.path.getsize(tmp_out_path) > 0:
                        list_files.append(tmp_out_path)
                        break
                    else:
                        pass
#                         print(f'Attemp {attempt+1} : Failed to download {ftp_path}.')
        
        #if 
        elif os.path.exists(dst_path)==True:
            if os.path.getsize(dst_path) > 0:
                continue
            else:
                os.remove(dst_path)
                for attempt in range(0, 10):
                    ftp_download(ftp_conn, ftp_path, tmp_out_path, skip_exist=True)
                    if os.path.getsize(tmp_out_path) > 0:
                        list_files.append(tmp_out_path)
                        break
                    else:
                        pass
#                         print(f'Attemp {attempt+1} : Failed to download {ftp_path}.')
                                       
        #print(ftp_path)

    if count_days % copy_frequency == 0:
        for filepath in list_files:
            dst_path = filepath.replace(TMP_FOLDER, MAIN_FOLDER)
            if os.path.exists(dst_path) == False:
                dst_folder = os.path.dirname(dst_path)
                os.makedirs(dst_folder, exist_ok=True)            
                shutil.copyfile(filepath, dst_path)
#                 print(f'{filepath} has been copied to {dst_path}.')
        list_files = []
    count_days += 1

550 /realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0000.02_AsiaSE.csv.zip: No such file or directory
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0000.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0000.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0000.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0000.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0000.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0000.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0000.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\20

550 /realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0700.02_AsiaSE.csv.zip: No such file or directory
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0700.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0700.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0700.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0700.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0700.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0700.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.0700.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\20

550 /realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.1400.02_AsiaSE.csv.zip: No such file or directory
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.1400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.1400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.1400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.1400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.1400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.1400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.1400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\20

550 /realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.2100.02_AsiaSE.csv.zip: No such file or directory
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.2100.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.2100.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.2100.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.2100.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.2100.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.2100.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\01\gsmap_nrt.20210601.2100.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\20

550 /realtime_ver/v6/txt/02_AsiaSE\2021\06\02\gsmap_nrt.20210602.0400.02_AsiaSE.csv.zip: No such file or directory
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\02\gsmap_nrt.20210602.0400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\02\gsmap_nrt.20210602.0400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\02\gsmap_nrt.20210602.0400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\02\gsmap_nrt.20210602.0400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\02\gsmap_nrt.20210602.0400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\02\gsmap_nrt.20210602.0400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\02\gsmap_nrt.20210602.0400.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\20

...

550 /realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.1500.02_AsiaSE.csv.zip: No such file or directory
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.1500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.1500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.1500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.1500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.1500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.1500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.1500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\20

550 /realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.2200.02_AsiaSE.csv.zip: No such file or directory
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.2200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.2200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.2200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.2200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.2200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.2200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\24\gsmap_nrt.20210624.2200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\20

550 /realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.0500.02_AsiaSE.csv.zip: No such file or directory
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.0500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.0500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.0500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.0500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.0500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.0500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.0500.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\20

550 /realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1200.02_AsiaSE.csv.zip: No such file or directory
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1200.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\20

550 /realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1900.02_AsiaSE.csv.zip: No such file or directory
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1900.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1900.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1900.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1900.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1900.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1900.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\2021\06\25\gsmap_nrt.20210625.1900.02_AsiaSE.csv.zip already exists.
thaiwa/rainfall\tmp\realtime_ver/v6/txt/02_AsiaSE\20

## save to dataframe

In [ ]:
list_exist_files = list_files_re(download_folder, '.zip')
list_exist_files

[]

In [ ]:
df_exist_files_drive = pd.DataFrame(list_exist_files, columns=['file_path'])
df_exist_files_drive['file_nm'] = df_exist_files_drive['file_path'].apply(lambda x: os.path.basename(x))
df_exist_files_drive['date'] = df_exist_files_drive['file_nm'].str.slice(10, 18)
df_exist_files_drive['year'] = df_exist_files_drive['date'].str.slice(0, 4)
df_exist_files_drive['month'] = df_exist_files_drive['date'].str.slice(4, 6)
df_exist_files_drive['day'] = df_exist_files_drive['date'].str.slice(6, 8)
df_exist_files_drive['hour'] = df_exist_files_drive['file_nm'].str.slice(19, 23)
df_exist_files_drive

,file_path,file_nm,date,year,month,day,hour


In [ ]:
list_error = []
list_corrupted_files = []

for (cur_year, cur_date), df_curr in tqdm(df_exist_files_drive.groupby(['year', 'date'])):
    out_path = os.path.join(raw_folder, cur_year, f'gsmap_rt_v6_thailand_{cur_date}.parquet')

    if len(df_curr) != 24:
        list_error.append(f'{cur_date} has only {len(df_curr)} files.')
        continue

    elif os.path.exists(out_path)==False:
        os.makedirs(os.path.dirname(out_path), exist_ok=True)   
        df_curr = df_curr.sort_values(['hour'])
        
        list_df_gsmap = []
        try:
            for file_path in df_curr['file_path']:
                #load csv file
                df_gsmap = pd.read_csv(file_path, dtype=np.float64)
                
                df_gsmap.columns = ['lat', 'lon', 'rain_rate', 'calib_rain_rate']
                df_gsmap['rain_rate'] = df_gsmap['rain_rate'].astype(np.float32)
                df_gsmap['calib_rain_rate'] = df_gsmap['calib_rain_rate'].astype(np.float32)

                #add date & hour columns
                file_name = os.path.basename(file_path)
                date = datetime.datetime.strptime(file_name[10:18], '%Y%m%d')
                hour = np.int8(file_name[19:21])
                df_gsmap['date'] = date
                df_gsmap['hour'] = hour

                #Thailand's extent
                lon_min, lat_min, lon_max, lat_max = 96.95, 4.95, 105.85, 20.95

                #filter only Thailand's points
                df_gsmap = df_gsmap[(df_gsmap['lat'].between(lat_min, lat_max)) & (df_gsmap['lon'].between(lon_min, lon_max))].copy()

                #reindex columns
                df_gsmap = df_gsmap.reindex(columns=['date', 'hour', 'lat', 'lon', 'rain_rate', 'calib_rain_rate'])
                
                list_df_gsmap.append(df_gsmap)
            df_gsmap_daily = pd.concat(list_df_gsmap)
            df_gsmap_daily.to_parquet(out_path)
            print(f'{out_path} has been saved.')
        except Exception as err:
            for file_path in df_curr['file_path']:
                try:
                    df_gsmap = pd.read_csv(file_path, dtype=np.float64)                
                except Exception as err:
                    list_corrupted_files.append(file_path)                    
                    print(f'{file_path} is corrupted!')
                    if os.path.exists(file_path)==True:
                        os.remove(file_path)
                    list_error.append(f'{file_path} is corrupted!')
            
    else:
        print(f'{out_path} already exists.')
    